copy from https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [1]:
import numpy
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint

# 데이터

In [2]:
import string

all_char = string.ascii_lowercase + string.digits
# all_char = 'abcdefghijklmnopqrstuvwxyz0123456789'
# len(all_char) = 36

raw_text = all_char*100
# len(raw_text) = 3600

SEQ_LENGTH = 10

## 문자 인덱스 <--> 문자 변환

In [5]:
chars = sorted(list(set(raw_text)))
# chars = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

char_to_int = dict((c, i) for i, c in enumerate(chars))
# char_to_int = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, ... }
int_to_char = dict((i, c) for i, c in enumerate(chars))
# int_to_char = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'a', 11: 'b', 12: 'c', ... }

## text 길이, 문자 수

In [9]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  3600
Total Vocab:  36


## 입출력 데이터 만들기

```
# seq_length=20

         1         2         3         4         5         6    
123456789012345678901234567890123456789012345678901234567890
alice's adventures in wonderland
<----- seq_in------>
                    ^
                    |
                seq_out
```

```
seq_in = 'alice's adventures i'
seq_out = 'n'
```

In [10]:
seq_length = SEQ_LENGTH
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# X.shape = (148474, 100, 1)   # 데이터 148,474개, 1개 데이터는 길이 100 

# normalize
X = X / float(n_vocab)
y = numpy.array(dataY)
# y = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 31, 24, 42, 28, 38, 1, 22, 20, 37, 37 ... ]
# y.shape = (148474, )  # 데이터 148,474개. 

Total Patterns:  3590


In [11]:
print(y.shape)

(3590,)


# 학습

## 모델 정의

In [15]:

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

## 학습 실행

In [16]:
model.fit(X, y, epochs=50, batch_size=256)

Epoch 1/50
15/15 [==============================] - 7s 9ms/step - loss: 3.4963
Epoch 2/50
15/15 [==============================] - 0s 7ms/step - loss: 2.9719
Epoch 3/50
15/15 [==============================] - 0s 7ms/step - loss: 2.3492
Epoch 4/50
15/15 [==============================] - 0s 7ms/step - loss: 1.8684
Epoch 5/50
15/15 [==============================] - 0s 8ms/step - loss: 1.6444
Epoch 6/50
15/15 [==============================] - 0s 7ms/step - loss: 1.2970
Epoch 7/50
15/15 [==============================] - 0s 20ms/step - loss: 1.2344
Epoch 8/50
15/15 [==============================] - 0s 8ms/step - loss: 0.9682
Epoch 9/50
15/15 [==============================] - 0s 7ms/step - loss: 0.8686
Epoch 10/50
15/15 [==============================] - 0s 7ms/step - loss: 0.7430
Epoch 11/50
15/15 [==============================] - 0s 7ms/step - loss: 0.7137
Epoch 12/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6256
Epoch 13/50
15/15 [=============================

## 예측

In [17]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	print(result, end="")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" 789abcdefg "
hijklmnopqrstuvwxyz0123456789bccdffhiiklmmopprssuvwwyzz1223456789cccdffhijklmmopqrstuvwxyz0123456789
Done.


# GRU로


In [ ]:
from tensorflow.keras.layers import GRU

# define the LSTM model
model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(GRU(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
# model.add(LSTM(256))
model.add(GRU(256))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X, y, epochs=50, batch_size=256)

Epoch 1/50
15/15 [==============================] - 3s 6ms/step - loss: 3.4841
Epoch 2/50
15/15 [==============================] - 0s 6ms/step - loss: 2.7317
Epoch 3/50
15/15 [==============================] - 0s 6ms/step - loss: 2.0706
Epoch 4/50
15/15 [==============================] - 0s 6ms/step - loss: 1.6820
Epoch 5/50
15/15 [==============================] - 0s 6ms/step - loss: 1.4387
Epoch 6/50
15/15 [==============================] - 0s 6ms/step - loss: 1.2141
Epoch 7/50
15/15 [==============================] - 0s 6ms/step - loss: 1.0592
Epoch 8/50
15/15 [==============================] - 0s 6ms/step - loss: 1.1264
Epoch 9/50
15/15 [==============================] - 0s 6ms/step - loss: 1.0305
Epoch 10/50
15/15 [==============================] - 0s 6ms/step - loss: 0.8324
Epoch 11/50
15/15 [==============================] - 0s 6ms/step - loss: 0.7032
Epoch 12/50
15/15 [==============================] - 0s 6ms/step - loss: 0.6047
Epoch 13/50
15/15 [==============================

In [ ]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	print(result, end="")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" 3456789abc "
cdefghijklmnopqrstuvwwxzz0123455789aaacdefghijjkmnopqqrstuwwxyzz0123456789aaacdefghijjkmnopqqrstuwwx
Done.


# Bidirectional 로

In [ ]:
from tensorflow.keras.layers import Bidirectional

# define the LSTM model
model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Bidirectional(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True)))
model.add(Dropout(0.2))
# model.add(LSTM(256))
model.add(Bidirectional(LSTM(256)))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X, y, epochs=50, batch_size=256)

Epoch 1/50
15/15 [==============================] - 6s 13ms/step - loss: 3.4383
Epoch 2/50
15/15 [==============================] - 0s 13ms/step - loss: 2.4754
Epoch 3/50
15/15 [==============================] - 0s 13ms/step - loss: 1.5842
Epoch 4/50
15/15 [==============================] - 0s 13ms/step - loss: 1.1516
Epoch 5/50
15/15 [==============================] - 0s 13ms/step - loss: 0.8496
Epoch 6/50
15/15 [==============================] - 0s 13ms/step - loss: 0.7526
Epoch 7/50
15/15 [==============================] - 0s 13ms/step - loss: 0.6296
Epoch 8/50
15/15 [==============================] - 0s 13ms/step - loss: 0.5683
Epoch 9/50
15/15 [==============================] - 0s 13ms/step - loss: 0.4108
Epoch 10/50
15/15 [==============================] - 0s 13ms/step - loss: 0.4342
Epoch 11/50
15/15 [==============================] - 0s 13ms/step - loss: 0.3621
Epoch 12/50
15/15 [==============================] - 0s 13ms/step - loss: 0.3792
Epoch 13/50
15/15 [==================

In [ ]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	print(result, end="")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" hijklmnopq "
rstuvwxyz0123456789abcdefghijklmnopqrstuvwxyz0123456789abcdefghijklmnopqrstuvwxyz0123456789abcdefghi
Done.
